In [ ]:
#Repulsion energy 
with open("ion_ion.dat","r") as repulsion:
    repulsion_energy = float(repulsion.readline().strip())

#print("Ion-ion Electrostatic Energy :", repulsion_energy)

In [ ]:
#One electron integrals
import numpy as np
#open and read the file
with open("one_electron_integrals.dat", "r") as oei:
    first_line = oei.readline().strip().split()
    noccp = int(first_line[0]) #number of occupied spatial orbitals
    nvirt = int(first_line[1]) #number of virtual orbitals
    
    #initialize the dictionary to store the tensor
    one_electron_integrals = {}

    #Read the integrals line by line 
    for line in oei:
        data = line.strip().split()
        if len(data) == 3:
            i, j, value = int(data[0]), int(data[1]), float(data[2])
            one_electron_integrals[(i,j)] = value 
            one_electron_integrals[(j,i)] = value #ensure H(i,j) = H(j,i)
#Total Spatial Orbitals
norb = noccp + nvirt

#Construct spatial one-electron integral matrix
one_electron_tensor = np.zeros((norb, norb)) #Square matrix created

#Fill in the matrix 
for (i,j), value in one_electron_integrals.items():
    one_electron_tensor[i-1, j-1] = value #Adjusting to zero based indexing

    
#Construct block diagonal spin-orbital Hamiltonian 
num_spin_orb = 2 * norb #twice the size of spatial orbs 
one_electron_spin_tensor = np.zeros((num_spin_orb, num_spin_orb))

#Fill in the block diagonal spin-orbital tensor 
for p in range(norb):
    for q in range(norb):
        value = one_electron_tensor[p,q]

        #fill alpha-alpha and beta-beta blocks
        #alpha-alpha block
        one_electron_spin_tensor[p,q] = value
        #beta-beta block
        one_electron_spin_tensor[p + norb, q + norb] = value

#print number of Occupied and Virtual Orbitals
print(f"Number of occupied spatial orbitals : {noccp}")
print(f"Number of virtual spatial orbitals : {nvirt}")

#Print One-electron integral tensor 
print("One electron integral tensor :")
print(one_electron_tensor)

#Print spin one-electron integral tensor
print("\nSpin-Orbital One-Electron Integral Tensor (Block Diagonal):")
#print(one_electron_spin_tensor.shape)
print(one_electron_spin_tensor)


In [ ]:
#Two electron integrals (Coulomb and Exchange)
import numpy as np
#Function to read and symmetrize integrals 
def construct_tei(filename):
    with open(filename, "r") as tei:
        #read first line 
        noccp, nvirt, value2 = map(int, tei.readline().split())
        norb = noccp + nvirt  # Total number of orbitals
        tensor = np.zeros((norb, norb, norb, norb))
        
        #Read tensors
        for _ in range(value2):
            p, q, r, s, val = map(float, tei.readline().split())
            p, q, r, s = int(p)-1, int(q)-1, int(r)-1, int(s)-1 #Adjust zero indexing 

            #store values in all equivalent positions 
            for a, b, c, d in [
                (p,q,r,s),(p,q,s,r),(q,p,r,s),(q,p,s,r),
                (r,s,p,q),(r,s,q,p),(s,r,p,q),(s,r,q,p)
                
            ]:
                tensor[(a,b,c,d)] = val
    return tensor


# Process and save Coulomb tensor
coulomb_tensor = construct_tei("two_electron_integrals_coulomb.dat")
print("coulomb tensor", coulomb_tensor)
print("Coulomb Tensor Shape:",coulomb_tensor.shape)

# Process and save Exchange tensor
exchange_tensor = construct_tei("two_electron_integrals_exchange.dat")
print("exchange tensor", exchange_tensor)
print("Exchange Tensor Shape:",exchange_tensor.shape)
